# Simulating IPFS and IPNS Systems

This notebook provides a Python-based simulation of IPFS (InterPlanetary File System) and IPNS (InterPlanetary Naming System) to test various linking strategies for storing and retrieving IPAROs.

The notebook uses three classes to simulate these systems:
- **IPARO**: Represents the storage object on IPFS.
- **IPNS**: Keeps track of the latest capture for different websites.
- **IPFS**: Simulates the hashing, storage, and retrieval of IPARO objects.

The goal of the simulation is to test various linking strategies.

In [3]:
# Importing the necessary libraries
import hashlib
import random

## IPARO Object

**Properties:**
- `CID`: The CID (Content Identifier) generated by IPFS.
- `Data`: The data of the capture.
- `Linked Node CID(s)`: The CID(s) of the nodes linked to it.

**Functions:**
- `get_cid`: Returns the CID of the IPARO.
- `get_linked_cids`: Returns the CID(s) of the linked node(s).
- `get_content`: Returns the content of the IPARO.
- `__str__`: Returns a string representation of the IPARO object.

In [4]:
class IPARO:
    def __init__(self, cid: str, linked_cids: list, content: str, timestamp: str):
        """
        Initialize an IPARO object with its CID, linked CID(s), and content.

        Args:
            cid (str): The CID of the IPARO.
            linked_cids (list): List of CIDs of linked nodes.
            content (str): The content of the IPARO.
        """
        self.__cid = cid
        self.__linked_cids = linked_cids
        self.__content = content
        self.__timestamp = timestamp

    def get_cid(self) -> str:
        '''
        Returns the CID of the IPARO.

        Returns:
            str: The CID of the IPARO.
        '''
        return self.__cid

    def get_linked_cids(self) -> list:
        '''
        Returns the CID(s) of linked nodes.

        Returns:
            list: List of linked node CIDs.
        '''
        return self.__linked_cids

    def get_content(self) -> str:
        '''
        Returns the content of the IPARO.

        Returns:
            str: The content stored in the IPARO.
        '''
        return self.__content

    def get_timestamp(self) -> str:
        '''
        Returns the timestamp of the IPARO

        Returns:
            str: The timestamp in seconds since the epoch
        '''
        return self.__timestamp

    def __str__(self):
        '''
        Returns a string representation of the IPARO object.

        Returns:
            str: A string containing the CID, linked CID(s), and content of the IPARO.
        '''
        iparo = {
            "CID": self.__cid,
            "Content": self.__content,
            "Linked CID(s)": self.__linked_cids,
            "Timestamp": self.__timestamp,
        }
        return str(iparo)

## IPNS Object

**Description:**
- The IPNS class stores and maps the latest CID of a website.
- Tracks the number of operations (get and update) performed.

**Functions:**
- `update`: Updates the latest CID of a website.
- `get_cid`: Retrieves the CID of the latest capture for a website.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [5]:
class IPNS:
    def __init__(self):
        """
        Initialize the IPNS object with an empty hashmap for storing CIDs 
        and counters for tracking operations.
        """
        self.data = {}
        self.update_count = 0
        self.get_count = 0

    def update(self, url, cid):
        '''
        Updates the latest CID for a given URL.

        Args:
            url (str): The URL of the website.
            cid (str): The CID of the latest capture.
        '''
        self.update_count += 1
        self.data[url] = cid

    def get_cid(self, url) -> str:
        '''
        Retrieves the latest CID for a given URL.

        Args:
            url (str): The URL of the website.

        Returns:
            str: The CID of the latest capture for the given URL.
        '''
        self.get_count += 1
        return self.data[url]

    def get_counts(self) -> dict:
        '''
        Returns the number of update and get operations performed.

        Returns:
            dict: Dictionary with the counts of update and get operations.
        '''
        counts = {"get": self.get_count, "update": self.update_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.update_count = 0
        self.get_count = 0

## IPFS Object

**Description:**
- The IPFS class stores the nodes and simulates the hashing, storage, and retrieval operations.
- Tracks the number of operations (hash, store, retrieve).

**Functions:**
- `hash`: Hashes the content of a node to generate its CID.
- `store`: Stores a node with its CID.
- `retrieve`: Retrieves a node using its CID.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [6]:
class IPFS:
    def __init__(self):
        '''
        Initialize the IPFS object with an empty hashmap for storing nodes
        and counters for tracking operations.
        '''
        self.data = {}
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def hash(self, content: str) -> str:
        '''
        Hashes the content to generate a CID.

        Args:
            content (str): The content of the node.

        Returns:
            str: The generated CID.
        '''
        sha256_hash = hashlib.sha256(content.encode()).hexdigest()
        self.hash_count += 1
        return 'Qm' + sha256_hash[:34]

    def store(self, cid: str, node: IPARO):
        '''
        Stores a node with its CID.

        Args:
            cid (str): The CID of the node.
            node (IPARO): The IPARO object to store.
        '''
        self.store_count += 1
        self.data[cid] = node

    def retrieve(self, cid) -> IPARO:
        '''
        Retrieves a node using its CID.

        Args:
            cid (str): The CID of the node to retrieve.

        Returns:
            IPARO: The retrieved IPARO object.
        '''
        self.retrieve_count += 1
        return self.data[cid]

    def get_counts(self) -> dict:
        '''
        Returns the number of hash, store, and retrieve operations performed.

        Returns:
            dict: Dictionary with counts of hash, store, and retrieve operations.
        '''
        counts = {"hash": self.hash_count, "store": self.store_count,
                  "retrieve": self.retrieve_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def reset_data(self):
        self.data = {}

    def get_data(self) -> dict:
        """Returns the data stored by IPFS (for debugging)."""
        return self.data

## Initialization and Operation Tracking

Here, we initialize the IPFS and IPNS objects and define a helper function `get_op_counts()` to display the number of operations performed.


In [7]:
# Initializing the simulated IPFS and IPNS
ipfs = IPFS()
ipns = IPNS()


def get_op_counts():
    '''
    Displays the number of operations performed by IPNS and IPFS.
    '''
    print("Number of operations IPNS performed:")
    print(ipns.get_counts())
    print("Number of operations IPFS performed:")
    print(ipfs.get_counts())

## Testing Different Linking Strategies

### 1. Linking to Only the Previous Node

In this test, each node will link only to the previous node in the chain. This strategy will be used to simulate a simple sequential storage system.


#### Storing Nodes

In [7]:
import time

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)


# Automate the creation of additional nodes
for i in range(1, NODE_NUM):
    content = f"Node {i}"
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    linked_cids = [ipns.get_cid(URL)]  # Link to the previous node
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 99, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 0}


#### Retrieving Nodes

The following section tests retrieval of nodes by simulating a random node search.


In [8]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Traverse back through the linked nodes to find the target
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
while node.get_content() != target_content:
    node = ipfs.retrieve(node.get_linked_cids()[0])

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 84
Found node: {'CID': 'Qmf15c3dd334e396dab32613b810e0dd5411', 'Content': 'Node 84', 'Linked CID(s)': ['Qmafea068ad0feb1c593650b0d3c1452e63f'], 'Timestamp': 1737428201.730081}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 16}


### 2. Linking to all previous nodes

In this test, each node will link to all the previous nodes in the chain.

#### Storing Nodes

In [9]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

From here, there are 2 ways of creating a new node for this linking strategy, since this is a simulation, no data corruption can happen but that might not be true in practice. When retrieving the latest node which should contain the CIDs of all the previous node, two scenarios can happen:
1. The data is intact and the CIDs in the list is "correct" (which we really can't know for sure) and we can just add it to the new node we're creating
2. The data is corrupt and one or more of the CIDs is wrong or unfinished, in which case we have to recheck every CID to rebuild a new list of linked CIDs (not to mention fixing all the corrupted nodes)

So, for the purpose of this simulation, we will perform a check for every CID in the linked CID list of an IPARO to simulate the worst case scenario every time

In [10]:
# To automate adding the rest of the nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    linked_cids = []
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = latest_node_linked_cids
    for link_cid in latest_node_linked_cids:
        ipfs.retrieve(link_cid)
        # Checking and repairing nodes goes here
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

# print(ipfs.get_data())
get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 4965}


In this worst case scenario, where we have to retrieve and verify every CIDs in the linked CIDs of an IPARO, the retrieve count goes to almost 5000 (if we're storing 100 nodes)\
Of course this trade off makes it really easy to navigate to all the nodes just from the latest nodes

#### Retrieving nodes

The following section tests retrieval of nodes by simulating a random node search.

In [11]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Traverse back through the linked nodes to find the target
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
for linked_cid in linked_cids:
    node = ipfs.retrieve(linked_cid)
    if node.get_content() == target_content:
        break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 8
Found node: {'CID': 'Qm4b370f9f479108d86bae0c5083c4352ed2', 'Content': 'Node 8', 'Linked CID(s)': ['Qm4216cab2473d49f85d47188cf8f75bf86e', 'Qm3629c5fa56928016adad762f5fa0066b36', 'Qme0f38e8d7cee5b83ed74849a45f3c6da7c', 'Qm5167407f305494ee5f01fefa6a92869da3', 'Qmd3789b271c074193f18de6c620c2ca6cb7', 'Qme93df0b19a6079bf58111a4aad3a14ea21', 'Qm3550d07d4b8c7180692600c3efe2154ccc', 'Qm3d6062452025f7395fabd83c4e943eb734', 'Qm4b370f9f479108d86bae0c5083c4352ed2', 'Qm0bf767fa22586ba90e0c24279e74c245ba', 'Qmb58861925798d9948b36e2b3ecd2be3997', 'Qm59654c6677a51e75afaec19cea6bfd435b', 'Qm7b9c215421aaff7aa67c4b68d1c3080fea', 'Qm25deb5ffceb5fbee71ec9192728b8eccc0', 'Qm9f15daf0883b3abc8eee3ec25e6b1a8953', 'Qm3acab236879df227988afc20ec1187b209', 'Qm73650d3d5338bacbc65c62eed15adb2ee6', 'Qm5378d3b347e8e3a796df224aa42f5d1583', 'Qm847305ec14412e61c66a2d6734b8851063', 'Qmff0e9a5b9c79397f5a6bc7cfde81811501', 'Qmed5e9bff7d80ed8f052c5a7ae254e145f3', 'Qm703eb553a3d69f76d968

### 3. Linking to previous and first node

In this test, each node will link to the previous node and the first node in the chain.

#### Storing Nodes

In [12]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

In [13]:
# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    linked_cids = []
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids.append(latest_node_linked_cids[0])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

# print(ipfs.get_data())
get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 108}


#### Retrieving Nodes

In [14]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    while True:
        node = ipfs.retrieve(linked_cids[1])
        linked_cids = node.get_linked_cids()
        if node.get_content() == target_content:
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 45
Found node: {'CID': 'Qm950b11f4402c765264a5a6dcf9d6648e54', 'Content': 'Node 45', 'Linked CID(s)': ['Qm5f5860c31e69674345f10a21d0350f5b7d', 'Qm2404a4c85c84706bb8cec4156ae083ef12'], 'Timestamp': 1737428201.801247}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 56}


### 4. Linking to K-previous and first node

In this test, each node will link to K previous node and the first node in the chain.

#### Storing Nodes

In [15]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"
K = 5

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

In [16]:
# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []
    linked_cids.append(latest_node_linked_cids[0])
    if len(latest_node_linked_cids) == K+1:
        linked_cids.extend(latest_node_linked_cids[2:])
    else:
        linked_cids.extend(latest_node_linked_cids[1:])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 154}


#### Retrieving Nodes

In [17]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    # run through the rest of the linked cids list to check
    for cid in linked_cids:
        node = ipfs.retrieve(cid)
        if node.get_content() == target_content:
            break
    # Get the linked cids of the node at index 1
    node = ipfs.retrieve(linked_cids[1])
    linked_cids = node.get_linked_cids()
    while True:
        # Flag to determine if we should exit the while loop
        found = False

        # Run through the rest of the linked CIDs list to check
        for cid in linked_cids[1:]:
            node = ipfs.retrieve(cid)
            if node.get_content() == target_content:
                found = True
                break

        if found:
            break  # Exit the while loop if the node was found

        if len(linked_cids) >= 2:
            node = ipfs.retrieve(linked_cids[1])
            linked_cids = node.get_linked_cids()
        else:
            print('Can\'t find node')
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 43
Found node: {'CID': 'Qmd04ec15c93d8df72525a6e5226ee11a9ef', 'Content': 'Node 43', 'Linked CID(s)': ['Qm491546fff686a7def2238cd7cb2634d9ca', 'Qmd797044ad4c25c12c23fa917d2e04dd0a3', 'Qm5956949dcfbc84c27a2ace8f8395482f55', 'Qma0a87a40865da181e95141593dee5cd30d', 'Qm07a5defdfaa6423a74ccf57e62313f3d12', 'Qm72c31096e3a3774c28bb354edbdf455a6c'], 'Timestamp': 1737428201.824037}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 74}


### 5. Linking to K-random and first node

In this test, each node will link to a random K previous node and the first node in the chain.

#### Storing Nodes

In [18]:
import random

# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 30
URL = "example.com"
Kmin = 5
Kmax = 10

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []
    linked_cids.append(latest_node_linked_cids[0])
    K = random.randint(Kmin, Kmax)
    # Check if the number of linked CIDs is greater than K and add K-1 random linked CIDs
    if len(latest_node_linked_cids) > K:
        linked_cids.extend(random.sample(latest_node_linked_cids[1:], K-1))
    # If the number of linked CIDs is less than K add all the linked CIDs
    else:
        linked_cids.extend(latest_node_linked_cids[1:])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    print("K = ", K+1)
    print("Length of linked_cids: ", len(linked_cids))
    print(node)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

K =  6
Length of linked_cids:  2
{'CID': 'Qm750d68949cfdc86823f33ae432887df3ab', 'Content': 'Node 2', 'Linked CID(s)': ['Qm1eaacbddecf989bec254059533e4849115', 'Qm4b4f1ab270304851bb4fe17a5414157019'], 'Timestamp': 1737428201.8384829}
K =  9
Length of linked_cids:  3
{'CID': 'Qmc27ea7587b0a15c91c73ebb8a78eda6d8d', 'Content': 'Node 3', 'Linked CID(s)': ['Qm1eaacbddecf989bec254059533e4849115', 'Qm4b4f1ab270304851bb4fe17a5414157019', 'Qm750d68949cfdc86823f33ae432887df3ab'], 'Timestamp': 1737428201.8385642}
K =  11
Length of linked_cids:  4
{'CID': 'Qm1204356a2c370af9d8f06af52b09cc648e', 'Content': 'Node 4', 'Linked CID(s)': ['Qm1eaacbddecf989bec254059533e4849115', 'Qm4b4f1ab270304851bb4fe17a5414157019', 'Qm750d68949cfdc86823f33ae432887df3ab', 'Qmc27ea7587b0a15c91c73ebb8a78eda6d8d'], 'Timestamp': 1737428201.838586}
K =  10
Length of linked_cids:  5
{'CID': 'Qme56607d8dd59d5ccadd65a255903298f2d', 'Content': 'Node 5', 'Linked CID(s)': ['Qm1eaacbddecf989bec254059533e4849115', 'Qm4b4f1ab2703048

#### Retrieving nodes

In [19]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    # run through the rest of the linked cids list to check
    for cid in linked_cids:
        node = ipfs.retrieve(cid)
        if node.get_content() == target_content:
            break
    # Get the linked cids of the node at index 1
    node = ipfs.retrieve(linked_cids[1])
    linked_cids = node.get_linked_cids()
    while True:
        # Flag to determine if we should exit the while loop
        found = False

        # Run through the rest of the linked CIDs list to check
        for cid in linked_cids[1:]:
            node = ipfs.retrieve(cid)
            if node.get_content() == target_content:
                found = True
                break

        if found:
            break  # Exit the while loop if the node was found

        if len(linked_cids) >= 2:
            node = ipfs.retrieve(linked_cids[1])
            linked_cids = node.get_linked_cids()
        else:
            print('Can\'t find node')
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 7
Found node: {'CID': 'Qma5e868b96f476293a453f4efd4cda3b668', 'Content': 'Node 7', 'Linked CID(s)': ['Qm1eaacbddecf989bec254059533e4849115', 'Qm4b4f1ab270304851bb4fe17a5414157019', 'Qm750d68949cfdc86823f33ae432887df3ab', 'Qmc27ea7587b0a15c91c73ebb8a78eda6d8d', 'Qm1204356a2c370af9d8f06af52b09cc648e', 'Qme56607d8dd59d5ccadd65a255903298f2d', 'Qmc0d352288eedbad18718f6a8d80d67d0b0'], 'Timestamp': 1737428201.838634}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 19}


### Linking to Sequential Exponential (Base K, K an integer)

#### Storing nodes

In [20]:
ipfs.reset_data()

# Testing parameters
NODE_NUM = 50
K = 2
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node " + str(i)
    print(content)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []

    # Link to previous node FIRST
    linked_cids.append(latest_node_linked_cids[0])
    cids = []

    j = 1
    # Temp node = latest node CID inserted (before this one). Call that node K-1.
    if i == 4:
        pass
    temp_node = latest_node
    print(temp_node.get_content())
    done = False
    while True:
        # Theorem: For any m < len(temp_node_linked_cids), the (1+m)th-to-last position of the linked CIDs list will link
        # K^m nodes away from the node.
        # Base Case: The previous node always gets assigned the last position (m=0), so the base case holds.
        # Inducive Case: Suppose that the property holds for m=k. Then, we need to prove that it holds for m=k+1.
        # The start node is K^k nodes away from the most recent node. But if we use the kth position of the linked
        # CIDs, and travelled the kth link (K - 1) times, then the CID to be added to the linked CIDs (at the (k+1)th
        # position) is K^k + (K-1) * K^k = K*K^k = K^(k+1), which proves the inductive case.
        # Therefore, by the Principle of Mathematical Induction, this theorem holds.
        for _ in range(K - 1):
            temp_cid = temp_node.get_cid()
            temp_node_linked_cids = temp_node.get_linked_cids()
            done = j > len(temp_node_linked_cids)
            if done:
                break
            temp_node = ipfs.retrieve(temp_node_linked_cids[-j])
        if done:
            break
        print(temp_node.get_content())
        # Ensure no duplicate links
        if temp_node.get_cid() not in linked_cids:
            cids.append(temp_node.get_cid())
        j += 1
    cids = list(reversed(cids))
    linked_cids.extend(cids)
    linked_cids.append(latest_node_cid)

    print("Length of linked_cids: ", len(linked_cids))
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Node 2
Node 1
Node 0
Length of linked_cids:  2
Node 3
Node 2
Node 1
Length of linked_cids:  3
Node 4
Node 3
Node 2
Node 0
Length of linked_cids:  3
Node 5
Node 4
Node 3
Node 1
Length of linked_cids:  4
Node 6
Node 5
Node 4
Node 2
Length of linked_cids:  4
Node 7
Node 6
Node 5
Node 3
Node 0
Length of linked_cids:  4
Node 8
Node 7
Node 6
Node 4
Node 0
Length of linked_cids:  4
Node 9
Node 8
Node 7
Node 5
Node 1
Length of linked_cids:  5
Node 10
Node 9
Node 8
Node 6
Node 2
Length of linked_cids:  5
Node 11
Node 10
Node 9
Node 7
Node 3
Length of linked_cids:  5
Node 12
Node 11
Node 10
Node 8
Node 4
Length of linked_cids:  5
Node 13
Node 12
Node 11
Node 9
Node 5
Node 0
Length of linked_cids:  5
Node 14
Node 13
Node 12
Node 10
Node 6
Node 0
Length of linked_cids:  5
Node 15
Node 14
Node 13
Node 11
Node 7
Node 0
Length of linked_cids:  5
Node 16
Node 15
Node 14
Node 12
Node 8
Node 0
Length of linked_cids:  5
Node 17
Node 16
Node 15
Node 13
Node 9
Node 1
Length of linked_cids:  6
Node 18
Node 

In [21]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    # run through the rest of the linked cids list to check
    for cid in linked_cids:
        node = ipfs.retrieve(cid)
        if node.get_content() == target_content:
            break
    # Get the linked cids of the node at index -1 (where the latest index is)
    node = ipfs.retrieve(linked_cids[-1])
    linked_cids = node.get_linked_cids()
    while True:
        # Flag to determine if we should exit the while loop
        found = False

        # Run through the rest of the linked CIDs list to check
        for cid in linked_cids[1:]:
            node = ipfs.retrieve(cid)
            if node.get_content() == target_content:
                found = True
                break

        if found:
            print(f"Found node: {node}")
            break  # Exit the while loop if the node was found

        if len(linked_cids) >= 2:
            node = ipfs.retrieve(linked_cids[-1])
            linked_cids = node.get_linked_cids()
            # Output the found node
        else:
            print('Can\'t find node')
            break

get_op_counts()

Looking for node with content: Node 43
Found node: {'CID': 'Qma0282b9775f62039594198da49584e0504', 'Content': 'Node 43', 'Linked CID(s)': ['Qm67039aefbd1eefaacc2ab2c15ed3c1ee6b', 'Qm12ad975b7ce5bfb75f1cac68c6de3a517a', 'Qmfaa1f3a5a859b842a2dbd7c732ab38975b', 'Qmdd614f221da76afbf8c8946343832376b1', 'Qm00697b064736271a2f931a1b0648e33f8e', 'Qm60edecfdddf4f81c450814137369036739', 'Qmb58b781c73b66ef5dd74ee8b8bb502e459'], 'Timestamp': 1737428201.857858}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 21}


### 7. Linking to sequentially uniform N-prior

In [21]:
ipfs.reset_data()

import time
import math

# Testing parameters
NODE_NUM = 100
URL = "example.com"
N = 10
i = 0
num_links = max(1, math.floor(NODE_NUM / N))
temp_cids = []

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)
temp_cids.append(cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)
temp_cids.append(cid)

print(f"Node {1} created with Linked CIDs: {linked_cids}")

# Automate adding the rest of the nodes
for i in range(2, NODE_NUM):
    content = "Node"+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)

    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []

    # Link to previous node 
    linked_cids.append(latest_node_linked_cids[0])
    linked_cids.append(latest_node_cid)

    if i % num_links == 0:
        linked_cid = temp_cid
        temp_cids = []

    # Create the new node
    new_node = IPARO(cid=cid, linked_cids=linked_cids,
                     content=content, timestamp=timestamp)
    ipfs.store(cid, new_node)
    ipns.update(URL, cid)
    temp_cids.append(cid)

    print(f"Node {i} created with Linked CIDs: {linked_cids}")

# Final output
get_op_counts() 

Node 1 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f']
Node 2 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f', 'Qma2c2108ab057959c31d8e32ea623ca5fb5']
Node 3 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f', 'Qm4a156ef84d394fbf84d658fca7209e77ee']
Node 4 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f', 'Qm41e8f624cff4284c8e68828007f154ab58']
Node 5 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f', 'Qm86f2485d256637a61fc39e2ce565cd181b']
Node 6 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f', 'Qmc65a1c456cc135092a335cd29687c7530a']
Node 7 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f', 'Qmc0a1572f68e4da75465f955b6f04b2d34d']
Node 8 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f', 'Qmf3be44e286d818b7f5e6230d8478a9a33d']
Node 9 created with Linked CIDs: ['Qm67ffef3f493bf9c5cb558ba2af1d9dd00f', 'Qm90bc525d0aae0de8c83306db2e4b198e5e']
['Qm67ffef3f49

In [20]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node{node_num}"
print(f"Looking for node with content: {target_content}")

# Retrieve the starting node
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()

# Traverse the graph using a set to avoid revisiting nodes
visited_cids = set()
found = False

while linked_cids and not found:
    next_cids = []
    for cid in linked_cids:
        if cid in visited_cids:
            continue  # Skip already visited nodes
        visited_cids.add(cid)
        node = ipfs.retrieve(cid)
        print(f"Visiting CID: {cid}, Content: {node.get_content()}")  # Debug log
        if node.get_content() == target_content:
            found = True
            break
        # Add new links to the next search queue
        next_cids.extend(node.get_linked_cids())
    linked_cids = next_cids

if found:
    print(f"Found node: {node}")
else:
    print(f"Can't find node with content: {target_content}")
    all_contents = [ipfs.retrieve(cid).get_content() for cid in visited_cids]
    print(f"Available contents: {all_contents}")

# Output the operation counts
get_op_counts()


Looking for node with content: Node53
Visiting CID: Qma5341bb36e0854dfa1eae06a0d8cbcf0a4, Content: Node 0
Visiting CID: Qmfe79e41ab8ff1f499ff84a2418b47b0d03, Content: Node98
Visiting CID: Qm690533c2a93c4a4e5b7f92ff0552471bf3, Content: Node97
Visiting CID: Qmd87cac04bb7b8ec042328edc243f4bec78, Content: Node96
Visiting CID: Qm2389b78c109a46492a1bb4895ec3d3db6f, Content: Node95
Visiting CID: Qmfd5ddf598130d6404d8991a33b83f1f4ce, Content: Node94
Visiting CID: Qm8dd8413bdd12b2bec3d5ba994a2cdb7813, Content: Node93
Visiting CID: Qm42eba717e92d5e0cddc4810ab7a9323907, Content: Node92
Visiting CID: Qm99738a0ca9c4087fb1af02b942356a38bb, Content: Node91
Visiting CID: Qm2b776e5aa9144384e5299816aec34ce905, Content: Node90
Visiting CID: Qmacba1cb6762d2186988ea1c64a36cf4cc6, Content: Node89
Visiting CID: Qme1b6d0724f0ec2fbc83e70909700892469, Content: Node88
Visiting CID: Qmab5be3ad45efe5106979e857ac16b8fc41, Content: Node87
Visiting CID: Qm947534dd2b4aee5ac834406de8347e4956, Content: Node86
Visiting C

### Other strategies to be tested